# QuickStart even better

This is a modified version of quickstart from pytorch

We're going to use several techniques such as the following:
+ Capacity adjustment
+ Early Stopping
+ Layer, Batch Normalization
+ Dropout

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

In [4]:
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


we could learn that 
+ number of data used for training the model: 64
+ channel: 1
+ height: 28
+ width: 28

Now, let's make a model!

## Capacity Adjustment

Capacity Adjustment could be done in two ways
+ make it **wider** (recommended for linear relationships)
+ make it **deeper** (recommended for non-linear relationships)

In [5]:
device = "cpu"

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.LogSoftmax(dim=1)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
class NeuralNetwork_Wider(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
class NeuralNetwork_deeper(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model_vanilla = NeuralNetwork().to(device)
model_wider = NeuralNetwork_Wider().to(device)
model_deeper = NeuralNetwork_deeper().to(device)
modelList = []
modelList.append(model_vanilla)
modelList.append(model_wider)
modelList.append(model_deeper)

we need to optimize the model parameters via training

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [11]:
for model in modelList:
    print(model)
    loss_fn = nn.CrossEntropyLoss() # we are solving classification problems
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # use basic stochastic gradient descent
    epochs = 5
    for t in range(epochs):
        print(f"Epoch {t+1}\n")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): LogSoftmax(dim=1)
  )
)
Epoch 1

loss: 2.303284  [   64/60000]
loss: 2.291668  [ 6464/60000]
loss: 2.268611  [12864/60000]
loss: 2.264872  [19264/60000]
loss: 2.249413  [25664/60000]
loss: 2.222299  [32064/60000]
loss: 2.223895  [38464/60000]
loss: 2.193138  [44864/60000]
loss: 2.196489  [51264/60000]
loss: 2.164532  [57664/60000]
Test Error: 
 Accuracy: 53.9%, Avg loss: 2.151114 

Epoch 2

loss: 2.162143  [   64/60000]
loss: 2.152901  [ 6464/60000]
loss: 2.085797  [12864/60000]
loss: 2.106697  [19264/60000]
loss: 2.065038  [25664/60000]
loss: 2.002006  [32064/60000]
loss: 2.028186  [38464/60000]
loss: 1.951716  [44864/60000]
loss: 1.964999  [51264/60000]
loss: 1.895547

By training the vanilla model from the [link](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html), we got $1.1$ of AVG loss

Now we need to save the model

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
